In [1]:
import onnx
from onnx2pytorch import ConvertModel
from auto_LiRPA import BoundedModule, BoundedTensor, PerturbationLpNorm

from vnnlib.compat import read_vnnlib_simple

import torch
import numpy as np

from collections import OrderedDict

import csv

import os
from timeit import default_timer as timer

# Loading ONNX and VNNLib Specifications

In [2]:
def load_onnx_model(onnx_path, input_shape):
    onnx_model = onnx.load(onnx_path)
    torch_model = ConvertModel(onnx_model)
    
    x_concrete = torch.zeros(input_shape)
    model = BoundedModule(torch_model, x_concrete)
    return model

In [3]:
def load_vnnlib_spec(vnnlib_path, input_shape, n_out):
    n_in = np.prod(input_shape)
    res = read_vnnlib_simple(vnnlib_path, n_in, n_out)
    bnds, spec = res[0]
    
    bnds = np.array(bnds)
    lbs = bnds[:,0]
    ubs = bnds[:,1]
    
    data_min = torch.tensor(lbs, dtype=torch.float32).reshape(input_shape)
    data_max = torch.tensor(ubs, dtype=torch.float32).reshape(input_shape)
    center = 0.5*(data_min + data_max)

    ptb = PerturbationLpNorm(x_L=data_min, x_U=data_max)
    x = BoundedTensor(center, ptb)
    
    return x

In [11]:
%cd /home/ufuk/Documents/Programming/ICNN_verification

/home/ufuk/Documents/Programming/ICNN_verification


In [21]:
onnx_path = 'script/Experiments/mnist-net_256x4.onnx'
vnnlib_path = 'script/Experiments/prop_0_spiral_25.vnnlib'

In [22]:
model = load_onnx_model(onnx_path, [1,1,1,784])
x = load_vnnlib_spec(vnnlib_path, [1,1,1,784], 10)

/home/ufuk/miniconda3/envs/autolirpa_icnn/lib/python3.10/site-packages/onnx2pytorch/convert/model.py:163: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if not self.experimental and inputs[0].shape[self.batch_dim] > 1:
/home/ufuk/miniconda3/envs/autolirpa_icnn/lib/python3.10/site-packages/auto_LiRPA/parse_graph.py:154: FutureWarning: 'torch.onnx.symbolic_helper._set_opset_version' is deprecated in version 1.13 and will be removed in 2.0. Please remove its usage and avoid setting internal variables directly.
  _set_opset_version(12)


# Helper Methods

In [23]:
def get_layers(model):
    return [l for l in model.nodes() if l.perturbed]

In [24]:
def get_intermediate_bounds(model):
    """
    Returns a dictionary containing the concrete lower and upper bounds of each layer.
    
    Implemented own method to filter out bounds for weight matrices.
    
    Only call this method after compute_bounds()!
    """
    od = OrderedDict()
    for l in get_layers(model):
        if hasattr(l, 'lower'):
            od[l.name] = (l.lower, l.upper)
            
    return od

# Get Intermediate Bounds

In [25]:
model.compute_bounds(x=(x,), method='ibp')

(tensor([[-447.2314, -373.2225, -492.4267, -520.7729, -479.3799, -425.8518,
          -413.6321, -447.6174, -532.3359, -408.0772]], grad_fn=<AddBackward0>),
 tensor([[356.4533, 318.0782, 320.3527, 274.7251, 371.1610, 338.5822, 395.8803,
          377.8232, 375.9247, 400.2977]], grad_fn=<AddBackward0>))

In [26]:
bounds_dict = get_intermediate_bounds(model)
for k, (lbs, ubs) in bounds_dict.items():
    print(f"{k}: {lbs.shape}")

/0: torch.Size([1, 1, 1, 784])
/21: torch.Size([1, 784])
/input: torch.Size([1, 256])
/23: torch.Size([1, 256])
/input.3: torch.Size([1, 256])
/25: torch.Size([1, 256])
/input.7: torch.Size([1, 256])
/27: torch.Size([1, 256])
/input.11: torch.Size([1, 256])
/29: torch.Size([1, 256])
/30: torch.Size([1, 10])


In [27]:
bounds_dict['/30']

(tensor([[-447.2314, -373.2225, -492.4267, -520.7729, -479.3799, -425.8518,
          -413.6321, -447.6174, -532.3359, -408.0772]], grad_fn=<AddBackward0>),
 tensor([[356.4533, 318.0782, 320.3527, 274.7251, 371.1610, 338.5822, 395.8803,
          377.8232, 375.9247, 400.2977]], grad_fn=<AddBackward0>))

In [28]:
model.compute_bounds(x=(x,), method='crown')

(tensor([[-0.4672, -0.4103, -0.4026, -0.3573, -0.4667, -0.4770, -0.3886, -0.4031,
          -0.5523, -0.4308]], grad_fn=<ViewBackward0>),
 tensor([[0.6201, 0.4692, 0.6307, 0.4833, 1.8194, 0.5396, 0.9479, 0.4172, 0.6483,
          0.6467]], grad_fn=<ViewBackward0>))

In [29]:
bounds_dict['/30']

(tensor([[-447.2314, -373.2225, -492.4267, -520.7729, -479.3799, -425.8518,
          -413.6321, -447.6174, -532.3359, -408.0772]], grad_fn=<AddBackward0>),
 tensor([[356.4533, 318.0782, 320.3527, 274.7251, 371.1610, 338.5822, 395.8803,
          377.8232, 375.9247, 400.2977]], grad_fn=<AddBackward0>))

In [30]:
bounds_dict_crown = get_intermediate_bounds(model)
bounds_dict_crown['/30']

(tensor([[-0.4672, -0.4103, -0.4026, -0.3573, -0.4667, -0.4770, -0.3886, -0.4031,
          -0.5523, -0.4308]], grad_fn=<ViewBackward0>),
 tensor([[0.6201, 0.4692, 0.6307, 0.4833, 1.8194, 0.5396, 0.9479, 0.4172, 0.6483,
          0.6467]], grad_fn=<ViewBackward0>))

**Attention**: CROWN-bounds are only saved for pre-activation nodes and the output!
(in contrast to interval propagation bounds, that are saved for every layer)

In [31]:
bounds_dict_crown.keys()

odict_keys(['/0', '/input', '/input.3', '/input.7', '/input.11', '/30'])

In [33]:
lbs11_ibp, ubs11_ibp = bounds_dict['/input.11']
lbs11_crown, ubs11_crown = bounds_dict_crown['/input.11']

print(lbs11_ibp[:,:10])
print(lbs11_crown[:,:10])

tensor([[-171.4623, -160.4540, -304.7760, -128.2996, -153.7493, -117.9111,
         -123.5061, -278.8083, -158.6687, -207.4034]],
       grad_fn=<SliceBackward0>)


KeyboardInterrupt: 

# Sampling via CROWN

In order to use CROWN to calculate bounds for the sampled directions, we make use of the possibility to supply
- a constraint matrix (which we use to represent the sampled directions) and
- to specify the output layer (which we just set to the layer, for which we want to sample)

The shape of the constraint matrix is `(batch, n_directions, n_neurons)`, where we just set `batch=1`.

The output layer is specified via the node names in the node dictionary.

In [4]:
n_batch = 1
n_dirs = 5
n_neurons = 256
C = torch.randn(n_batch, n_dirs, n_neurons)

model.compute_bounds(x=(x,), final_node_name='/input.11', C=C, method='crown')

NameError: name 'model' is not defined

We can also use $\alpha$-CROWN to optimize the bounds of the directions.

In [17]:
model.compute_bounds(x=(x,), final_node_name='/input.11', C=C, method='alpha-crown')

(tensor([[  13.3784,  -36.1162,  -60.9758, -128.4353,  -66.3043]]),
 tensor([[ 76.5662,  -7.1403,  -8.4030, -21.8835, -14.1343]]))

When using more iterations, the bounds may get slightly better.

In [18]:
model.bound_opts

{'conv_mode': 'patches',
 'sparse_intermediate_bounds': True,
 'sparse_conv_intermediate_bounds': True,
 'sparse_intermediate_bounds_with_ibp': True,
 'sparse_features_alpha': True,
 'sparse_spec_alpha': True,
 'minimum_sparsity': 0.9,
 'enable_opt_interm_bounds': False,
 'crown_batch_size': inf,
 'forward_refinement': False,
 'dynamic_forward': False,
 'forward_max_dim': 1000000000,
 'use_full_conv_alpha': True,
 'disabled_optimization': [],
 'use_full_conv_alpha_thresh': 512,
 'verbosity': 0,
 'optimize_graph': {'optimizer': None},
 'optimize_bound_args': {'enable_alpha_crown': True,
  'enable_beta_crown': False,
  'apply_output_constraints_to': None,
  'iteration': 20,
  'use_shared_alpha': False,
  'optimizer': 'adam',
  'keep_best': True,
  'fix_interm_bounds': True,
  'lr_alpha': 0.5,
  'lr_beta': 0.05,
  'lr_cut_beta': 0.005,
  'init_alpha': True,
  'lr_coeffs': 0.01,
  'intermediate_refinement_layers': [-1],
  'loss_reduction_func': <function auto_LiRPA.utils.<lambda>(x)>,
  's

In [19]:
def set_params(model, use_shared_alpha=False, iteration=20, early_stop_patience=10):
    model.bound_opts['optimize_bound_args']['use_shared_alpha'] = use_shared_alpha
    model.bound_opts['optimize_bound_args']['iteration'] = iteration
    model.bound_opts['optimize_bound_args']['early_stop_patience'] = early_stop_patience

In [20]:
set_params(model, iteration=100)
model.compute_bounds(x=(x,), final_node_name='/input.11', C=C, method='alpha-crown')

(tensor([[  13.4349,  -36.1024,  -60.9277, -128.4019,  -66.2657]]),
 tensor([[ 76.5378,  -7.1574,  -8.4328, -21.9471, -14.1642]]))

# Building LP Model

There is at least some code available to build LP and MILP models, but it doesn't seem to be maintained/is broken now. Maybe we can repair and use it.

In [89]:
model.build_solver_module(model_type='lp')

AttributeError: 'function' object has no attribute 'CONTINUOUS'

In [86]:
import gurobipy as grb

In [88]:
model.model = grb.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-27
